# Step 1: Importing Essential Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

# Step 2: Loading data and Making labels

In [2]:
#Dataset used: https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000
image_path = "archive/hmnist_28_28_RGB.csv"
meta_path = "archive/HAM10000_metadata.csv"

In [3]:
df = pd.read_csv(image_path)
metadata = pd.read_csv(meta_path)

In [4]:
df.tail()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
10010,183,165,181,182,165,180,184,166,182,188,...,208,185,187,208,186,186,206,187,189,0
10011,2,3,1,38,33,32,121,104,103,132,...,96,79,76,24,23,21,3,4,1,0
10012,132,118,118,167,149,149,175,156,160,184,...,204,181,178,181,159,153,172,151,145,0
10013,160,124,146,164,131,152,167,127,146,169,...,185,162,167,184,157,166,185,162,172,0
10014,175,142,121,181,150,134,181,150,133,178,...,159,79,82,174,137,125,175,139,126,6


In [5]:
metadata.tail()

,lesion_id,image_id,dx,dx_type,age,sex,localization
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face
10014,HAM_0003521,ISIC_0032258,mel,histo,70.0,female,back


In [6]:
#drop NA values
meta_set = metadata[["age", "sex", "localization"]]
df = pd.concat([meta_set, df], axis = 1)
df = df.dropna()
df.tail()

,age,sex,localization,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
10010,40.0,male,abdomen,183,165,181,182,165,180,184,...,208,185,187,208,186,186,206,187,189,0
10011,40.0,male,abdomen,2,3,1,38,33,32,121,...,96,79,76,24,23,21,3,4,1,0
10012,40.0,male,abdomen,132,118,118,167,149,149,175,...,204,181,178,181,159,153,172,151,145,0
10013,80.0,male,face,160,124,146,164,131,152,167,...,185,162,167,184,157,166,185,162,172,0
10014,70.0,female,back,175,142,121,181,150,134,181,...,159,79,82,174,137,125,175,139,126,6


# Step 3: Train Test Split

In [7]:
# Split into train, validation, and test.
np.random.seed(2070404)

# Shuffle all records.
df_shuffle = df.sample(frac = 1)

# Create split counts.
splits = np.multiply(len(df_shuffle), (0.6,0.2,0.2)).astype(int)
print(f"Split counts (train/ validation/ test): {splits}")

# Create split data sets.
train_set, valid_set, test_set = np.split(df_shuffle, [splits[0], splits[0] + splits[1]])

# Reset split set indicies.
train_set.reset_index(drop = True, inplace = True)
valid_set.reset_index(drop = True, inplace = True)
test_set.reset_index(drop = True, inplace = True)

Split counts (train/ validation/ test): [5974 1991 1991]


In [8]:
#separate features and labels
y_train_clean = train_set['label']
x_train_clean = train_set.drop(columns=['label'])

y_valid = valid_set['label']
x_valid = valid_set.drop(columns=['label'])

y_test = test_set['label']
x_test = test_set.drop(columns=['label'])

columns = list(x_train_clean)

# Step 4: Preprocessing

In [9]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#create a new train set that over-sample the minority class(es) by picking samples at random with replacement
oversample = RandomOverSampler()
x_train_oversample, y_train_oversample  = oversample.fit_resample(x_train_clean, y_train_clean)

#create encoder to encode categorical features as one-hot arrays
ohe = OneHotEncoder()
#create scaler to standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()

#apply encoder and scaler on clean (original) dataset's metadata
categorical_data_clean = ohe.fit_transform(x_train_clean[['sex', 'localization']]).toarray()
numerical_data_clean = scaler.fit_transform(x_train_clean[['age']].values)
train_tabular_clean = np.concatenate([categorical_data_clean, numerical_data_clean], axis = 1)
x_train_clean = x_train_clean.iloc[:,3:]

#apply encoder and scaler on oversample dataset's metadata
categorical_data_oversample = ohe.transform(x_train_oversample[['sex', 'localization']]).toarray()
numerical_data_oversample = scaler.transform(x_train_oversample[['age']].values)
train_tabular_oversample = np.concatenate([categorical_data_oversample, numerical_data_oversample], axis = 1)
x_train_oversample = x_train_oversample.iloc[:,3:]

#apply encoder and scaler on validation dataset's metadata
categorical_data_valid = ohe.transform(x_valid[['sex', 'localization']]).toarray()
numerical_data_valid = scaler.transform(x_valid[['age']].values)
valid_tabular_set = np.concatenate([categorical_data_valid, numerical_data_valid], axis = 1)
x_valid = x_valid.iloc[:,3:]


In [10]:
print(train_tabular_clean.shape)
print(x_train_clean.shape)

(5974, 19)
(5974, 2352)


In [11]:
print(train_tabular_oversample.shape)
print(x_train_oversample.shape)

(27965, 19)
(27965, 2352)


In [12]:
print(valid_tabular_set.shape)
print(x_valid.shape)

(1991, 19)
(1991, 2352)


In [13]:
import matplotlib.pyplot as plt
import random

x_train_clean = np.array(x_train_clean, dtype=np.uint8).reshape(-1,28,28,3)
x_train_oversample = np.array(x_train_oversample, dtype=np.uint8).reshape(-1,28,28,3)
x_valid = np.array(x_valid, dtype=np.uint8).reshape(-1,28,28,3)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow import keras
import tensorflow as tf

In [15]:
x_valid_augm = x_valid / 255.0

def aug_image(x_train_set, tabular_set, y_train_set, contrast_factor = 2, delta = 0.1, flip_flag = True):
    """ 
    apply transformaions and augmentations
    
    Args:
    x_train_set: dataset to be augmented and transformed
    tabular_set: metadata of the dataset
    y_train_set: labels of the dataset
    contrast_factor: contrast factor to be used
    delta: delta value to be used
    flip_flag: whether to apply flipping transformation to the dataset
    
    
    Returns:
    x_train_set_augm: transformed and augmented dataset
    tabular_set_augm: metadata of the transformed and augmented dataset
    y_train_set_augm: labels of the transformed and augmented dataset
    
    """
    
    
    tf.random.set_seed(1234)

    #transform image to greyscale
    x_train_set_augm = x_train_set / 255.0

    #change delta
    x_train_set_augm = tf.image.adjust_brightness(x_train_set_augm, delta = delta)

    #change contrast
    x_train_set_augm = tf.image.adjust_contrast(x_train_set_augm, contrast_factor = contrast_factor)

    #flip image
    if flip_flag:
        x_train_set_augm = tf.image.random_flip_left_right(x_train_set_augm)

    #concatenate original dataset and augmented dataset
    x_train_set_augm = tf.concat([x_train_set, x_train_set_augm], axis = 0)
    
    #concatenate original dataset metadata with itselt
    tabular_set_augm = tabular_set
    tabular_set_augm = tf.concat([tabular_set_augm, tabular_set], axis = 0)

    #concatenate original dataset labels with itselt
    y_train_set_augm = y_train_set
    y_train_set_augm = tf.concat([y_train_set, y_train_set_augm], axis = 0)


    #shuffle dataset
    shuffle = tf.random.shuffle(tf.range(tf.shape(x_train_set_augm)[0], dtype = tf.int32))
    x_train_set_augm = tf.gather(x_train_set_augm, shuffle)
    tabular_set_augm = tf.gather(tabular_set_augm, shuffle)
    y_train_set_augm = tf.gather(y_train_set_augm, shuffle).numpy() #also transforms y_train to numpy array

    #print(x_train_set_augm.shape)
    
    return x_train_set_augm, tabular_set_augm, y_train_set_augm

# Step 5: Model Building

### 5.1: build a neural network model for metadata features (tabular data)

In [16]:
def build_tabular_model(input_dim = 19, activation_func = "relu", tabular_dense_list = [64]):    
    """ 
    build a neural network model using keras
    
    Args:
    input_dim: dimension of the input
    activation_func: activation function to be used in the hidden dense layers
    tabular_dense_list: a list of dense layer units to be added
    
    Returns:
    model: A tf.keras model
    
    """
    
    model = tf.keras.Sequential()

    for i in range(len(tabular_dense_list)):
        #add dense layer 1
        if i == 0:
            model.add(tf.keras.layers.Dense(
                units = tabular_dense_list[i],
                activation = activation_func,
                input_dim = input_dim))
        #add additional dense layer(s)
        else:
            model.add(tf.keras.layers.Dense(
                units = tabular_dense_list[i],
                activation = activation_func))
            
    return model
    

### 5.2: build a CNN model for images

In [17]:
def build_cnn_model(kernel_size = 2, pool_size = 2):
    """
    Build a CNN model using Keras.

    Args:
    kernel_size: convolution layer kernel size
    pool_size: pooling layer pool size

    Returns:
    model: A tf.keras model
    """
    
    model = tf.keras.Sequential()

    #add convolution layer 1
    model.add(tf.keras.layers.Conv2D(
        filters = 32, kernel_size = (kernel_size, kernel_size),
        strides=(1,1), padding='same',
        data_format = 'channels_last',
        input_shape = (28, 28, 3),  # Updated input shape for RGB
        name='conv_1', activation='relu'))

    #add pooling layer 1
    model.add(tf.keras.layers.MaxPool2D(
        pool_size = (pool_size, pool_size), name = 'pool_1'))

    #add convolution layer 2
    model.add(tf.keras.layers.Conv2D(
        filters = 64, kernel_size = (kernel_size, kernel_size),
        strides = (1,1), padding = 'same',
        name = 'conv_2', activation = 'relu'))

    #add pooling layer 2
    model.add(tf.keras.layers.MaxPool2D(
        pool_size = (pool_size, pool_size), name = 'pool_2'))

    #add flattening layer
    model.add(tf.keras.layers.Flatten())


    #add dense layer 1
    model.add(tf.keras.layers.Dense(
        units = 1024, name = 'fc_1', 
        activation = 'relu'))

    #dropout
    model.add(tf.keras.layers.Dropout(rate = 0.5))
    
    #add dense layer 2
    model.add(tf.keras.layers.Dense(
        units = 1024, name = 'dense2', 
        activation = 'relu'))
                
    return model

### 5.3: build a mixed model combining the neural network for tabular data and the CNN for images

In [18]:
def build_mixed_model(kernel_size = 2, pool_size = 2, tabular_shape = 19, tabular_activation_func = "relu", tabular_dense_list = [64], mixed_dense_list = [1024]):
    """
    Build a mixed model using Keras.

    Args:
    kernel_size: convolution layer kernel size
    pool_size: pooling layer pool size
    tabular_shape: dimension of the input
    tabular_activation_func: activation function to be used in the hidden dense layers
    tabular_dense_list: a list of dense layer units to be added to the tabular NN model
    mixed_dense_list: a list of dense layer units to be added to the mixed model

    Returns:
    model: A tf.keras model
    """
    
    #clear previous sessions
    tf.keras.backend.clear_session()

    #initialize two models
    cnn_model = build_cnn_model(kernel_size = kernel_size, pool_size = pool_size)
    tabular_model = build_tabular_model(input_dim = tabular_shape, activation_func = tabular_activation_func, tabular_dense_list = tabular_dense_list)

    #concatenate the two models by adding a concatenation layer
    combined_input = tf.keras.layers.concatenate([cnn_model.output, tabular_model.output])
    
    curr_layer = combined_input

    #add more dense layers based on dense_list
    for i in mixed_dense_list:
        curr_layer = tf.keras.layers.Dense(i, activation = 'relu')(curr_layer)
    
    #add output layer
    output = tf.keras.layers.Dense(7, activation = 'softmax')(curr_layer) 

    model = tf.keras.Model(inputs = [cnn_model.input, tabular_model.input], outputs = output)

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), 
                  loss = 'sparse_categorical_crossentropy',  
                  metrics = ['accuracy'])
    
    return model

# Step 6: Fitting the Model Using Different Datasets

In [19]:
#create a list of dense layer unit lists for mixed model
mixed_dense_lists = [[1024], [1024, 512], [1024, 512, 256], [1024, 512, 256, 128], [1024, 512, 256, 128, 64], [1024, 512, 256, 128, 64, 32]]

#create a list of dense layer unit lists for tabular NN model
tabular_dense_lists = [[32], [32, 64], [32, 64, 64], [32, 64, 64, 32]]

#create a list of activation functions to be tested
activation_list = ["relu", "tanh"]

### 6.1: fit models using clean (original) dataset

In [20]:

clean_dict = {}

for mixed_dense_list in mixed_dense_lists:
    for tabular_dense_list in tabular_dense_lists:
        for activation_func in activation_list:
        
            print("---mixed dense list: " + str(mixed_dense_list) + ", tabular dense list: " + str(tabular_dense_list) + ", activation: " + activation_func + "---")
    
            model = build_mixed_model(kernel_size = 3,
                                      pool_size = 3,
                                      tabular_shape = train_tabular_clean.shape[1],
                                      tabular_activation_func = activation_func,
                                      tabular_dense_list = tabular_dense_list,
                                      mixed_dense_list = mixed_dense_list)
    
            history = model.fit(
                [x_train_clean,
                 train_tabular_clean],
                y_train_clean,
                validation_data = ([x_valid,
                                    valid_tabular_set],
                                   y_valid),
                epochs = 10,  
                batch_size = 128
            )
    
            clean_dict[(len(mixed_dense_list), len(tabular_dense_list), activation_func)] = [history.history['accuracy'][-1], history.history['val_accuracy'][-1]]


---mixed dense list: [1024], tabular dense list: [32], activation: relu---
Epoch 1/10


2023-12-08 10:19:25.415511: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


47/47 [==============================] - 2s 29ms/step - loss: 10.8992 - accuracy: 0.6096 - val_loss: 0.8760 - val_accuracy: 0.7027
Epoch 2/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8737 - accuracy: 0.6950 - val_loss: 0.8624 - val_accuracy: 0.7137
Epoch 3/10
47/47 [==============================] - 1s 26ms/step - loss: 0.8067 - accuracy: 0.7139 - val_loss: 0.7692 - val_accuracy: 0.7263
Epoch 4/10
47/47 [==============================] - 1s 26ms/step - loss: 0.7737 - accuracy: 0.7201 - val_loss: 0.7326 - val_accuracy: 0.7343
Epoch 5/10
47/47 [==============================] - 1s 26ms/step - loss: 0.7586 - accuracy: 0.7278 - val_loss: 0.7421 - val_accuracy: 0.7358
Epoch 6/10
47/47 [==============================] - 1s 27ms/step - loss: 0.7172 - accuracy: 0.7349 - val_loss: 0.7144 - val_accuracy: 0.7398
Epoch 7/10
47/47 [==============================] - 1s 27ms/step - loss: 0.6987 - accuracy: 0.7419 - val_loss: 0.7589 - val_accuracy: 0.7333
Epoch 8/10
47/47 [=====

Epoch 1/10
47/47 [==============================] - 2s 33ms/step - loss: 5.1754 - accuracy: 0.6302 - val_loss: 0.8769 - val_accuracy: 0.7102
Epoch 2/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8287 - accuracy: 0.7101 - val_loss: 0.7967 - val_accuracy: 0.7172
Epoch 3/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7543 - accuracy: 0.7263 - val_loss: 0.7075 - val_accuracy: 0.7348
Epoch 4/10
47/47 [==============================] - 1s 27ms/step - loss: 0.7238 - accuracy: 0.7285 - val_loss: 0.6888 - val_accuracy: 0.7368
Epoch 5/10
47/47 [==============================] - 1s 29ms/step - loss: 0.6948 - accuracy: 0.7466 - val_loss: 0.6453 - val_accuracy: 0.7534
Epoch 6/10
47/47 [==============================] - 2s 36ms/step - loss: 0.6747 - accuracy: 0.7494 - val_loss: 0.6475 - val_accuracy: 0.7524
Epoch 7/10
47/47 [==============================] - 2s 33ms/step - loss: 0.6328 - accuracy: 0.7611 - val_loss: 0.6157 - val_accuracy: 0.7725
Epoch 8/10
47

---mixed dense list: [1024], tabular dense list: [32, 64], activation: relu---
Epoch 1/10
47/47 [==============================] - 2s 30ms/step - loss: 6.7893 - accuracy: 0.6217 - val_loss: 0.9149 - val_accuracy: 0.6976
Epoch 2/10
47/47 [==============================] - 1s 32ms/step - loss: 0.9032 - accuracy: 0.6952 - val_loss: 0.8427 - val_accuracy: 0.7112
Epoch 3/10
47/47 [==============================] - 2s 38ms/step - loss: 0.8710 - accuracy: 0.7005 - val_loss: 0.8330 - val_accuracy: 0.7157
Epoch 4/10
47/47 [==============================] - 1s 31ms/step - loss: 0.8455 - accuracy: 0.7064 - val_loss: 0.8220 - val_accuracy: 0.7167
Epoch 5/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8277 - accuracy: 0.7089 - val_loss: 0.8232 - val_accuracy: 0.7162
Epoch 6/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8287 - accuracy: 0.7056 - val_loss: 0.8189 - val_accuracy: 0.7017
Epoch 7/10
47/47 [==============================] - 1s 27ms/step - loss: 0.

...

Epoch 1/10
47/47 [==============================] - 2s 30ms/step - loss: 2.4728 - accuracy: 0.6110 - val_loss: 0.9207 - val_accuracy: 0.6816
Epoch 2/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8594 - accuracy: 0.7002 - val_loss: 0.7970 - val_accuracy: 0.7122
Epoch 3/10
47/47 [==============================] - 1s 29ms/step - loss: 0.8005 - accuracy: 0.7141 - val_loss: 0.7661 - val_accuracy: 0.7197
Epoch 4/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7798 - accuracy: 0.7238 - val_loss: 0.8016 - val_accuracy: 0.7087
Epoch 5/10
47/47 [==============================] - 1s 29ms/step - loss: 0.7757 - accuracy: 0.7208 - val_loss: 0.7435 - val_accuracy: 0.7363
Epoch 6/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7574 - accuracy: 0.7230 - val_loss: 0.7524 - val_accuracy: 0.7278
Epoch 7/10
47/47 [==============================] - 1s 31ms/step - loss: 0.7386 - accuracy: 0.7369 - val_loss: 0.7440 - val_accuracy: 0.7358
Epoch 8/10
47

---mixed dense list: [1024, 512, 256, 128, 64, 32], tabular dense list: [32, 64, 64], activation: relu---
Epoch 1/10
47/47 [==============================] - 2s 29ms/step - loss: 1.9907 - accuracy: 0.6011 - val_loss: 1.0070 - val_accuracy: 0.6735
Epoch 2/10
47/47 [==============================] - 1s 28ms/step - loss: 0.9403 - accuracy: 0.6711 - val_loss: 0.8475 - val_accuracy: 0.7132
Epoch 3/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8580 - accuracy: 0.7051 - val_loss: 0.8138 - val_accuracy: 0.7353
Epoch 4/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7920 - accuracy: 0.7206 - val_loss: 0.7586 - val_accuracy: 0.7293
Epoch 5/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7561 - accuracy: 0.7266 - val_loss: 0.7271 - val_accuracy: 0.7313
Epoch 6/10
47/47 [==============================] - 1s 29ms/step - loss: 0.7445 - accuracy: 0.7338 - val_loss: 0.7155 - val_accuracy: 0.7479
Epoch 7/10
47/47 [==============================

---mixed dense list: [1024, 512, 256, 128, 64, 32], tabular dense list: [32, 64, 64], activation: tanh---
Epoch 1/10
47/47 [==============================] - 2s 30ms/step - loss: 2.2429 - accuracy: 0.6277 - val_loss: 0.8993 - val_accuracy: 0.6906
Epoch 2/10
47/47 [==============================] - 1s 29ms/step - loss: 0.8569 - accuracy: 0.6985 - val_loss: 0.8549 - val_accuracy: 0.7067
Epoch 3/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8093 - accuracy: 0.7143 - val_loss: 0.7518 - val_accuracy: 0.7268
Epoch 4/10
47/47 [==============================] - 1s 29ms/step - loss: 0.7895 - accuracy: 0.7154 - val_loss: 0.7577 - val_accuracy: 0.7273
Epoch 5/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7673 - accuracy: 0.7225 - val_loss: 0.7387 - val_accuracy: 0.7348
Epoch 6/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7456 - accuracy: 0.7313 - val_loss: 0.7439 - val_accuracy: 0.7323
Epoch 7/10
47/47 [==============================

Epoch 1/10
47/47 [==============================] - 2s 29ms/step - loss: 2.6179 - accuracy: 0.5814 - val_loss: 1.0055 - val_accuracy: 0.6735
Epoch 2/10
47/47 [==============================] - 1s 27ms/step - loss: 0.9147 - accuracy: 0.6860 - val_loss: 0.8086 - val_accuracy: 0.7142
Epoch 3/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8409 - accuracy: 0.7082 - val_loss: 0.8013 - val_accuracy: 0.7197
Epoch 4/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8199 - accuracy: 0.7079 - val_loss: 0.7960 - val_accuracy: 0.7197
Epoch 5/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7792 - accuracy: 0.7184 - val_loss: 0.7795 - val_accuracy: 0.7212
Epoch 6/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7630 - accuracy: 0.7240 - val_loss: 0.7292 - val_accuracy: 0.7373
Epoch 7/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7481 - accuracy: 0.7283 - val_loss: 0.7267 - val_accuracy: 0.7469
Epoch 8/10
47

Epoch 1/10
47/47 [==============================] - 2s 29ms/step - loss: 1.6556 - accuracy: 0.6317 - val_loss: 1.0247 - val_accuracy: 0.6735
Epoch 2/10
47/47 [==============================] - 1s 31ms/step - loss: 0.9220 - accuracy: 0.6738 - val_loss: 0.7935 - val_accuracy: 0.7338
Epoch 3/10
47/47 [==============================] - 1s 28ms/step - loss: 0.8335 - accuracy: 0.7181 - val_loss: 0.7833 - val_accuracy: 0.7192
Epoch 4/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7969 - accuracy: 0.7166 - val_loss: 0.7701 - val_accuracy: 0.7152
Epoch 5/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7997 - accuracy: 0.7231 - val_loss: 0.7577 - val_accuracy: 0.7383
Epoch 6/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7744 - accuracy: 0.7303 - val_loss: 0.7420 - val_accuracy: 0.7283
Epoch 7/10
47/47 [==============================] - 1s 28ms/step - loss: 0.7495 - accuracy: 0.7317 - val_loss: 0.7468 - val_accuracy: 0.7318
Epoch 8/10
47

### 6.2: fit models using oversample dataset

In [22]:

oversample_dict = {}

for mixed_dense_list in mixed_dense_lists:
    for tabular_dense_list in tabular_dense_lists:
        for activation_func in activation_list:
        
            print("---mixed dense list: " + str(mixed_dense_list) + ", tabular dense list: " + str(tabular_dense_list) + ", activation: " + activation_func + "---")
    
            model = build_mixed_model(kernel_size = 3,
                                      pool_size = 3,
                                      tabular_shape = train_tabular_oversample.shape[1],
                                      tabular_activation_func = activation_func,
                                      tabular_dense_list = tabular_dense_list,
                                      mixed_dense_list = mixed_dense_list)
    
            history = model.fit(
                [x_train_oversample,
                 train_tabular_oversample],
                y_train_oversample,
                validation_data = ([x_valid,
                                    valid_tabular_set],
                                   y_valid),
                epochs=10,  
                batch_size=128
            )

            oversample_dict[(len(mixed_dense_list), len(tabular_dense_list), activation_func)] = [history.history['accuracy'][-1], history.history['val_accuracy'][-1]]


---mixed dense list: [1024], tabular dense list: [32], activation: relu---
Epoch 1/10
219/219 [==============================] - 6s 28ms/step - loss: 2.7221 - accuracy: 0.5263 - val_loss: 0.8728 - val_accuracy: 0.6570
Epoch 2/10
219/219 [==============================] - 6s 26ms/step - loss: 0.6638 - accuracy: 0.7555 - val_loss: 0.9332 - val_accuracy: 0.6479
Epoch 3/10
219/219 [==============================] - 6s 25ms/step - loss: 0.4721 - accuracy: 0.8273 - val_loss: 0.9177 - val_accuracy: 0.6539
Epoch 4/10
219/219 [==============================] - 6s 25ms/step - loss: 0.3585 - accuracy: 0.8711 - val_loss: 0.8772 - val_accuracy: 0.7117
Epoch 5/10
219/219 [==============================] - 6s 25ms/step - loss: 0.2867 - accuracy: 0.8988 - val_loss: 0.9639 - val_accuracy: 0.6946
Epoch 6/10
219/219 [==============================] - 5s 24ms/step - loss: 0.2351 - accuracy: 0.9149 - val_loss: 0.9626 - val_accuracy: 0.7152
Epoch 7/10
219/219 [==============================] - 5s 25ms/step 

Epoch 1/10
219/219 [==============================] - 6s 25ms/step - loss: 2.9108 - accuracy: 0.5143 - val_loss: 0.9750 - val_accuracy: 0.6243
Epoch 2/10
219/219 [==============================] - 5s 25ms/step - loss: 0.7884 - accuracy: 0.7031 - val_loss: 0.8735 - val_accuracy: 0.6781
Epoch 3/10
219/219 [==============================] - 5s 24ms/step - loss: 0.5768 - accuracy: 0.7870 - val_loss: 0.8473 - val_accuracy: 0.6791
Epoch 4/10
219/219 [==============================] - 5s 25ms/step - loss: 0.4551 - accuracy: 0.8333 - val_loss: 1.0139 - val_accuracy: 0.6494
Epoch 5/10
219/219 [==============================] - 5s 24ms/step - loss: 0.3454 - accuracy: 0.8773 - val_loss: 0.9266 - val_accuracy: 0.7042
Epoch 6/10
219/219 [==============================] - 5s 25ms/step - loss: 0.2921 - accuracy: 0.8951 - val_loss: 0.9490 - val_accuracy: 0.6821
Epoch 7/10
219/219 [==============================] - 6s 29ms/step - loss: 0.2300 - accuracy: 0.9201 - val_loss: 1.0153 - val_accuracy: 0.6901

Epoch 1/10
219/219 [==============================] - 6s 25ms/step - loss: 2.6348 - accuracy: 0.5488 - val_loss: 1.1062 - val_accuracy: 0.5836
Epoch 2/10
219/219 [==============================] - 6s 25ms/step - loss: 0.6045 - accuracy: 0.7720 - val_loss: 0.8641 - val_accuracy: 0.6765
Epoch 3/10
219/219 [==============================] - 5s 25ms/step - loss: 0.4248 - accuracy: 0.8440 - val_loss: 0.8902 - val_accuracy: 0.6911
Epoch 4/10
219/219 [==============================] - 5s 25ms/step - loss: 0.3168 - accuracy: 0.8867 - val_loss: 0.8443 - val_accuracy: 0.7157
Epoch 5/10
219/219 [==============================] - 6s 25ms/step - loss: 0.2623 - accuracy: 0.9063 - val_loss: 0.8607 - val_accuracy: 0.7288
Epoch 6/10
219/219 [==============================] - 5s 25ms/step - loss: 0.2258 - accuracy: 0.9199 - val_loss: 0.9837 - val_accuracy: 0.7228
Epoch 7/10
219/219 [==============================] - 5s 24ms/step - loss: 0.1962 - accuracy: 0.9286 - val_loss: 0.9892 - val_accuracy: 0.7243

Epoch 1/10
219/219 [==============================] - 6s 25ms/step - loss: 3.5171 - accuracy: 0.5379 - val_loss: 1.0512 - val_accuracy: 0.5856
Epoch 2/10
219/219 [==============================] - 5s 24ms/step - loss: 0.6840 - accuracy: 0.7469 - val_loss: 0.8549 - val_accuracy: 0.6529
Epoch 3/10
219/219 [==============================] - 5s 24ms/step - loss: 0.4883 - accuracy: 0.8222 - val_loss: 0.8158 - val_accuracy: 0.6896
Epoch 4/10
219/219 [==============================] - 5s 24ms/step - loss: 0.3545 - accuracy: 0.8713 - val_loss: 1.0474 - val_accuracy: 0.6494
Epoch 5/10
219/219 [==============================] - 5s 24ms/step - loss: 0.2933 - accuracy: 0.8954 - val_loss: 0.9435 - val_accuracy: 0.6916
Epoch 6/10
219/219 [==============================] - 5s 24ms/step - loss: 0.2184 - accuracy: 0.9235 - val_loss: 1.0215 - val_accuracy: 0.7032
Epoch 7/10
219/219 [==============================] - 5s 24ms/step - loss: 0.1884 - accuracy: 0.9333 - val_loss: 0.9709 - val_accuracy: 0.7253

---mixed dense list: [1024], tabular dense list: [32, 64, 64], activation: relu---
Epoch 1/10
219/219 [==============================] - 5s 24ms/step - loss: 4.1096 - accuracy: 0.4683 - val_loss: 1.0827 - val_accuracy: 0.5741
Epoch 2/10
219/219 [==============================] - 5s 24ms/step - loss: 0.7833 - accuracy: 0.7046 - val_loss: 0.9443 - val_accuracy: 0.6323
Epoch 3/10
219/219 [==============================] - 5s 24ms/step - loss: 0.5915 - accuracy: 0.7769 - val_loss: 1.0169 - val_accuracy: 0.6178
Epoch 4/10
219/219 [==============================] - 5s 24ms/step - loss: 0.4435 - accuracy: 0.8359 - val_loss: 0.9462 - val_accuracy: 0.6524
Epoch 5/10
219/219 [==============================] - 5s 23ms/step - loss: 0.3565 - accuracy: 0.8707 - val_loss: 1.1130 - val_accuracy: 0.6238
Epoch 6/10
219/219 [==============================] - 6s 25ms/step - loss: 0.2732 - accuracy: 0.9037 - val_loss: 0.9690 - val_accuracy: 0.6991
Epoch 7/10
219/219 [==============================] - 5s 24

...

Epoch 1/10
219/219 [==============================] - 6s 27ms/step - loss: 1.7792 - accuracy: 0.4262 - val_loss: 0.9936 - val_accuracy: 0.6012
Epoch 2/10
219/219 [==============================] - 6s 27ms/step - loss: 0.8625 - accuracy: 0.6700 - val_loss: 0.8347 - val_accuracy: 0.6745
Epoch 3/10
219/219 [==============================] - 6s 27ms/step - loss: 0.6445 - accuracy: 0.7570 - val_loss: 0.8477 - val_accuracy: 0.6615
Epoch 4/10
219/219 [==============================] - 6s 27ms/step - loss: 0.4923 - accuracy: 0.8175 - val_loss: 0.9042 - val_accuracy: 0.6494
Epoch 5/10
219/219 [==============================] - 6s 27ms/step - loss: 0.4059 - accuracy: 0.8492 - val_loss: 0.8459 - val_accuracy: 0.6896
Epoch 6/10
219/219 [==============================] - 6s 27ms/step - loss: 0.3351 - accuracy: 0.8750 - val_loss: 1.0953 - val_accuracy: 0.6404
Epoch 7/10
219/219 [==============================] - 6s 27ms/step - loss: 0.2614 - accuracy: 0.9027 - val_loss: 0.9719 - val_accuracy: 0.6891

Epoch 1/10
219/219 [==============================] - 6s 27ms/step - loss: 1.7260 - accuracy: 0.3402 - val_loss: 1.1275 - val_accuracy: 0.5741
Epoch 2/10
219/219 [==============================] - 6s 26ms/step - loss: 0.9030 - accuracy: 0.6475 - val_loss: 1.0784 - val_accuracy: 0.5957
Epoch 3/10
219/219 [==============================] - 6s 27ms/step - loss: 0.6445 - accuracy: 0.7549 - val_loss: 0.8121 - val_accuracy: 0.6916
Epoch 4/10
219/219 [==============================] - 6s 26ms/step - loss: 0.4627 - accuracy: 0.8286 - val_loss: 0.9954 - val_accuracy: 0.6499
Epoch 5/10
219/219 [==============================] - 6s 26ms/step - loss: 0.3731 - accuracy: 0.8622 - val_loss: 0.8677 - val_accuracy: 0.7192
Epoch 6/10
219/219 [==============================] - 6s 26ms/step - loss: 0.2965 - accuracy: 0.8896 - val_loss: 1.0649 - val_accuracy: 0.6625
Epoch 7/10
219/219 [==============================] - 6s 26ms/step - loss: 0.2432 - accuracy: 0.9115 - val_loss: 1.0351 - val_accuracy: 0.6961

Epoch 1/10
219/219 [==============================] - 6s 26ms/step - loss: 1.4162 - accuracy: 0.4592 - val_loss: 1.1842 - val_accuracy: 0.5635
Epoch 2/10
219/219 [==============================] - 6s 26ms/step - loss: 0.7215 - accuracy: 0.7281 - val_loss: 0.8042 - val_accuracy: 0.7142
Epoch 3/10
219/219 [==============================] - 6s 25ms/step - loss: 0.5040 - accuracy: 0.8137 - val_loss: 0.8861 - val_accuracy: 0.6796
Epoch 4/10
219/219 [==============================] - 6s 25ms/step - loss: 0.3824 - accuracy: 0.8584 - val_loss: 0.9581 - val_accuracy: 0.6796
Epoch 5/10
219/219 [==============================] - 6s 26ms/step - loss: 0.3129 - accuracy: 0.8859 - val_loss: 0.8971 - val_accuracy: 0.6911
Epoch 6/10
219/219 [==============================] - 6s 26ms/step - loss: 0.2574 - accuracy: 0.9069 - val_loss: 0.8593 - val_accuracy: 0.7368
Epoch 7/10
219/219 [==============================] - 6s 25ms/step - loss: 0.2218 - accuracy: 0.9201 - val_loss: 1.0363 - val_accuracy: 0.7052

Epoch 1/10
219/219 [==============================] - 6s 27ms/step - loss: 1.7027 - accuracy: 0.4321 - val_loss: 1.2438 - val_accuracy: 0.5183
Epoch 2/10
219/219 [==============================] - 6s 27ms/step - loss: 0.8683 - accuracy: 0.6714 - val_loss: 1.0154 - val_accuracy: 0.6042
Epoch 3/10
219/219 [==============================] - 6s 27ms/step - loss: 0.6136 - accuracy: 0.7690 - val_loss: 0.8849 - val_accuracy: 0.6419
Epoch 4/10
219/219 [==============================] - 6s 27ms/step - loss: 0.4777 - accuracy: 0.8221 - val_loss: 1.1336 - val_accuracy: 0.6102
Epoch 5/10
219/219 [==============================] - 6s 27ms/step - loss: 0.3890 - accuracy: 0.8554 - val_loss: 0.9531 - val_accuracy: 0.6735
Epoch 6/10
219/219 [==============================] - 6s 28ms/step - loss: 0.3126 - accuracy: 0.8873 - val_loss: 0.9837 - val_accuracy: 0.6700
Epoch 7/10
219/219 [==============================] - 6s 27ms/step - loss: 0.2731 - accuracy: 0.9003 - val_loss: 0.9156 - val_accuracy: 0.7067

Epoch 1/10
219/219 [==============================] - 6s 27ms/step - loss: 1.3962 - accuracy: 0.4658 - val_loss: 1.0983 - val_accuracy: 0.6047
Epoch 2/10
219/219 [==============================] - 6s 26ms/step - loss: 0.7919 - accuracy: 0.7032 - val_loss: 0.8637 - val_accuracy: 0.6866
Epoch 3/10
219/219 [==============================] - 6s 26ms/step - loss: 0.6035 - accuracy: 0.7730 - val_loss: 0.9310 - val_accuracy: 0.6846
Epoch 4/10
219/219 [==============================] - 6s 27ms/step - loss: 0.4552 - accuracy: 0.8331 - val_loss: 0.9110 - val_accuracy: 0.6730
Epoch 5/10
219/219 [==============================] - 6s 26ms/step - loss: 0.3546 - accuracy: 0.8724 - val_loss: 0.9210 - val_accuracy: 0.7017
Epoch 6/10
219/219 [==============================] - 6s 27ms/step - loss: 0.2723 - accuracy: 0.8998 - val_loss: 0.9426 - val_accuracy: 0.7187
Epoch 7/10
219/219 [==============================] - 6s 27ms/step - loss: 0.2469 - accuracy: 0.9118 - val_loss: 1.2405 - val_accuracy: 0.6002

### 6.3: fit models using image augmentation dataset

In [23]:

image_augm_dict = {}

for mixed_dense_list in mixed_dense_lists:
    for tabular_dense_list in tabular_dense_lists:
        for activation_func in activation_list:
        
            print("---mixed dense list: " + str(mixed_dense_list) + ", tabular dense list: " + str(tabular_dense_list) + ", activation: " + activation_func + "---")
    
            x_train_augm, tabular_augm, y_train_augm = aug_image(x_train_clean,
                                                                 train_tabular_clean,
                                                                 y_train_clean,
                                                                 contrast_factor = 1,
                                                                 delta = 0.1,
                                                                 flip_flag = True)
    
            model = build_mixed_model(kernel_size = 3,
                                      pool_size = 3,
                                      tabular_shape = tabular_augm.shape[1],
                                      tabular_activation_func = activation_func,
                                      tabular_dense_list = tabular_dense_list,
                                      mixed_dense_list = mixed_dense_list)
    
            history = model.fit(
                [x_train_augm,
                 tabular_augm],
                y_train_augm,
                validation_data = ([x_valid_augm,
                                    valid_tabular_set],
                                   y_valid),
                epochs=10,  
                batch_size=128
            )

            image_augm_dict[(len(mixed_dense_list), len(tabular_dense_list), activation_func)] = [history.history['accuracy'][-1], history.history['val_accuracy'][-1]]
        

---mixed dense list: [1024], tabular dense list: [32], activation: relu---


Epoch 1/10
94/94 [==============================] - 2s 25ms/step - loss: 3.6590 - accuracy: 0.6614 - val_loss: 0.9333 - val_accuracy: 0.6941
Epoch 2/10
94/94 [==============================] - 2s 24ms/step - loss: 0.8820 - accuracy: 0.6990 - val_loss: 0.8895 - val_accuracy: 0.7047
Epoch 3/10
94/94 [==============================] - 2s 24ms/step - loss: 0.8549 - accuracy: 0.7010 - val_loss: 0.8367 - val_accuracy: 0.7127
Epoch 4/10
94/94 [==============================] - 2s 25ms/step - loss: 0.8374 - accuracy: 0.7083 - val_loss: 0.8294 - val_accuracy: 0.7152
Epoch 5/10
94/94 [==============================] - 2s 24ms/step - loss: 0.7982 - accuracy: 0.7098 - val_loss: 0.7478 - val_accuracy: 0.7283
Epoch 6/10
94/94 [==============================] - 2s 24ms/step - loss: 0.7541 - accuracy: 0.7204 - val_loss: 0.8194 - val_accuracy: 0.7167
Epoch 7/10
94/94 [==============================] - 2s 24ms/step - loss: 0.7380 - accuracy: 0.7268 - val_loss: 0.8657 - val_accuracy: 0.7233
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 25ms/step - loss: 2.8698 - accuracy: 0.6649 - val_loss: 0.9127 - val_accuracy: 0.6981
Epoch 2/10
94/94 [==============================] - 2s 24ms/step - loss: 0.8407 - accuracy: 0.7074 - val_loss: 0.8901 - val_accuracy: 0.7017
Epoch 3/10
94/94 [==============================] - 2s 24ms/step - loss: 0.7880 - accuracy: 0.7162 - val_loss: 0.7788 - val_accuracy: 0.7187
Epoch 4/10
94/94 [==============================] - 2s 24ms/step - loss: 0.7512 - accuracy: 0.7228 - val_loss: 0.7332 - val_accuracy: 0.7418
Epoch 5/10
94/94 [==============================] - 2s 23ms/step - loss: 0.7246 - accuracy: 0.7328 - val_loss: 0.7351 - val_accuracy: 0.7323
Epoch 6/10
94/94 [==============================] - 2s 24ms/step - loss: 0.7025 - accuracy: 0.7395 - val_loss: 0.8043 - val_accuracy: 0.7177
Epoch 7/10
94/94 [==============================] - 2s 24ms/step - loss: 0.6889 - accuracy: 0.7441 - val_loss: 0.7454 - val_accuracy: 0.7454
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 2s 25ms/step - loss: 2.5877 - accuracy: 0.6676 - val_loss: 0.9087 - val_accuracy: 0.7017
Epoch 2/10
94/94 [==============================] - 2s 23ms/step - loss: 0.8536 - accuracy: 0.7064 - val_loss: 0.8731 - val_accuracy: 0.6976
Epoch 3/10
94/94 [==============================] - 2s 24ms/step - loss: 0.8081 - accuracy: 0.7112 - val_loss: 0.8020 - val_accuracy: 0.7107
Epoch 4/10
94/94 [==============================] - 2s 24ms/step - loss: 0.7694 - accuracy: 0.7171 - val_loss: 0.7648 - val_accuracy: 0.7298
Epoch 5/10
94/94 [==============================] - 2s 25ms/step - loss: 0.7457 - accuracy: 0.7234 - val_loss: 0.7437 - val_accuracy: 0.7288
Epoch 6/10
94/94 [==============================] - 2s 24ms/step - loss: 0.7281 - accuracy: 0.7287 - val_loss: 0.7843 - val_accuracy: 0.7298
Epoch 7/10
94/94 [==============================] - 2s 24ms/step - loss: 0.7073 - accuracy: 0.7364 - val_loss: 0.7216 - val_accuracy: 0.7333
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 2s 24ms/step - loss: 1.9727 - accuracy: 0.6768 - val_loss: 0.9090 - val_accuracy: 0.6941
Epoch 2/10
94/94 [==============================] - 2s 24ms/step - loss: 0.8485 - accuracy: 0.7062 - val_loss: 0.8506 - val_accuracy: 0.7017
Epoch 3/10
94/94 [==============================] - 2s 23ms/step - loss: 0.7977 - accuracy: 0.7117 - val_loss: 0.7866 - val_accuracy: 0.7122
Epoch 4/10
94/94 [==============================] - 2s 23ms/step - loss: 0.7659 - accuracy: 0.7177 - val_loss: 0.7748 - val_accuracy: 0.7338
Epoch 5/10
94/94 [==============================] - 2s 23ms/step - loss: 0.7316 - accuracy: 0.7251 - val_loss: 0.7439 - val_accuracy: 0.7313
Epoch 6/10
94/94 [==============================] - 2s 23ms/step - loss: 0.7209 - accuracy: 0.7341 - val_loss: 0.7707 - val_accuracy: 0.7202
Epoch 7/10
94/94 [==============================] - 2s 24ms/step - loss: 0.6951 - accuracy: 0.7383 - val_loss: 0.7764 - val_accuracy: 0.7313
Epoch 8/10
94

...

Epoch 1/10
94/94 [==============================] - 3s 28ms/step - loss: 1.3594 - accuracy: 0.6566 - val_loss: 0.9643 - val_accuracy: 0.6866
Epoch 2/10
94/94 [==============================] - 3s 28ms/step - loss: 0.8862 - accuracy: 0.7010 - val_loss: 0.8894 - val_accuracy: 0.6981
Epoch 3/10
94/94 [==============================] - 3s 28ms/step - loss: 0.8462 - accuracy: 0.7057 - val_loss: 0.9070 - val_accuracy: 0.7042
Epoch 4/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7968 - accuracy: 0.7133 - val_loss: 0.7908 - val_accuracy: 0.7107
Epoch 5/10
94/94 [==============================] - 3s 29ms/step - loss: 0.7661 - accuracy: 0.7211 - val_loss: 0.7394 - val_accuracy: 0.7258
Epoch 6/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7445 - accuracy: 0.7238 - val_loss: 0.7806 - val_accuracy: 0.7228
Epoch 7/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7217 - accuracy: 0.7353 - val_loss: 0.7242 - val_accuracy: 0.7353
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 30ms/step - loss: 1.6046 - accuracy: 0.6443 - val_loss: 0.9820 - val_accuracy: 0.6735
Epoch 2/10
94/94 [==============================] - 3s 28ms/step - loss: 0.9136 - accuracy: 0.6927 - val_loss: 0.9022 - val_accuracy: 0.6961
Epoch 3/10
94/94 [==============================] - 3s 28ms/step - loss: 0.8433 - accuracy: 0.7081 - val_loss: 0.8599 - val_accuracy: 0.7117
Epoch 4/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7943 - accuracy: 0.7138 - val_loss: 0.7921 - val_accuracy: 0.7137
Epoch 5/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7723 - accuracy: 0.7191 - val_loss: 0.7542 - val_accuracy: 0.7318
Epoch 6/10
94/94 [==============================] - 3s 29ms/step - loss: 0.7525 - accuracy: 0.7261 - val_loss: 0.7838 - val_accuracy: 0.7363
Epoch 7/10
94/94 [==============================] - 3s 29ms/step - loss: 0.7491 - accuracy: 0.7308 - val_loss: 0.7520 - val_accuracy: 0.7433
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 29ms/step - loss: 1.3745 - accuracy: 0.6555 - val_loss: 0.9519 - val_accuracy: 0.6876
Epoch 2/10
94/94 [==============================] - 3s 29ms/step - loss: 0.8881 - accuracy: 0.6956 - val_loss: 0.8989 - val_accuracy: 0.7002
Epoch 3/10
94/94 [==============================] - 3s 28ms/step - loss: 0.8387 - accuracy: 0.7050 - val_loss: 0.8638 - val_accuracy: 0.7022
Epoch 4/10
94/94 [==============================] - 3s 28ms/step - loss: 0.8152 - accuracy: 0.7069 - val_loss: 0.8482 - val_accuracy: 0.7092
Epoch 5/10
94/94 [==============================] - 3s 27ms/step - loss: 0.8110 - accuracy: 0.7071 - val_loss: 0.7797 - val_accuracy: 0.7097
Epoch 6/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7713 - accuracy: 0.7154 - val_loss: 0.8241 - val_accuracy: 0.7187
Epoch 7/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7658 - accuracy: 0.7179 - val_loss: 0.7612 - val_accuracy: 0.7278
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 28ms/step - loss: 1.3774 - accuracy: 0.6485 - val_loss: 0.9828 - val_accuracy: 0.6735
Epoch 2/10
94/94 [==============================] - 3s 27ms/step - loss: 0.9278 - accuracy: 0.6832 - val_loss: 0.9289 - val_accuracy: 0.7047
Epoch 3/10
94/94 [==============================] - 3s 29ms/step - loss: 0.8675 - accuracy: 0.7057 - val_loss: 0.8764 - val_accuracy: 0.7107
Epoch 4/10
94/94 [==============================] - 3s 29ms/step - loss: 0.8169 - accuracy: 0.7132 - val_loss: 0.8280 - val_accuracy: 0.7142
Epoch 5/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7804 - accuracy: 0.7197 - val_loss: 0.8185 - val_accuracy: 0.7233
Epoch 6/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7621 - accuracy: 0.7230 - val_loss: 0.8436 - val_accuracy: 0.7207
Epoch 7/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7557 - accuracy: 0.7262 - val_loss: 0.8230 - val_accuracy: 0.7238
Epoch 8/10
94

Epoch 1/10
94/94 [==============================] - 3s 28ms/step - loss: 1.3572 - accuracy: 0.6576 - val_loss: 0.9493 - val_accuracy: 0.6976
Epoch 2/10
94/94 [==============================] - 3s 27ms/step - loss: 0.8773 - accuracy: 0.7021 - val_loss: 0.9620 - val_accuracy: 0.6986
Epoch 3/10
94/94 [==============================] - 3s 27ms/step - loss: 0.8194 - accuracy: 0.7104 - val_loss: 0.8254 - val_accuracy: 0.7062
Epoch 4/10
94/94 [==============================] - 3s 27ms/step - loss: 0.7928 - accuracy: 0.7133 - val_loss: 0.7671 - val_accuracy: 0.7122
Epoch 5/10
94/94 [==============================] - 3s 27ms/step - loss: 0.7617 - accuracy: 0.7210 - val_loss: 0.7806 - val_accuracy: 0.7137
Epoch 6/10
94/94 [==============================] - 3s 28ms/step - loss: 0.7418 - accuracy: 0.7274 - val_loss: 0.7347 - val_accuracy: 0.7398
Epoch 7/10
94/94 [==============================] - 2s 26ms/step - loss: 0.7381 - accuracy: 0.7287 - val_loss: 0.7190 - val_accuracy: 0.7313
Epoch 8/10
94

### 6.4: fit models using oversample + image augmentation dataset

In [24]:

image_augm_over_dict = {}

for mixed_dense_list in mixed_dense_lists:
    for tabular_dense_list in tabular_dense_lists:
        for activation_func in activation_list:
        
            print("---mixed dense list: " + str(mixed_dense_list) + ", tabular dense list: " + str(tabular_dense_list) + ", activation: " + activation_func + "---")

            x_train_oversample_augm, tabular_oversample_augm, y_train_oversample_augm = aug_image(x_train_oversample,
                                                                                                  train_tabular_oversample,
                                                                                                  y_train_oversample,
                                                                                                  contrast_factor = 1,
                                                                                                  delta = 0.1,
                                                                                                  flip_flag = True)
    
            model = build_mixed_model(kernel_size = 3,
                                      pool_size = 3,
                                      tabular_shape = tabular_oversample_augm.shape[1],
                                      tabular_activation_func = activation_func,
                                      tabular_dense_list = tabular_dense_list,
                                      mixed_dense_list = mixed_dense_list)
    
            history = model.fit(
                [x_train_oversample_augm,
                 tabular_oversample_augm],
                y_train_oversample_augm,
                validation_data = ([x_valid_augm,
                                    valid_tabular_set],
                                   y_valid),
                epochs=10,  
                batch_size=128
            )

            image_augm_over_dict[(len(mixed_dense_list), len(tabular_dense_list), activation_func)] = [history.history['accuracy'][-1], history.history['val_accuracy'][-1]]
        

---mixed dense list: [1024], tabular dense list: [32], activation: relu---


Epoch 1/10
437/437 [==============================] - 10s 23ms/step - loss: 1.5713 - accuracy: 0.5390 - val_loss: 0.8108 - val_accuracy: 0.6700
Epoch 2/10
437/437 [==============================] - 10s 23ms/step - loss: 0.7609 - accuracy: 0.7138 - val_loss: 0.7001 - val_accuracy: 0.7258
Epoch 3/10
437/437 [==============================] - 10s 23ms/step - loss: 0.5787 - accuracy: 0.7841 - val_loss: 0.7066 - val_accuracy: 0.7243
Epoch 4/10
437/437 [==============================] - 10s 23ms/step - loss: 0.4860 - accuracy: 0.8208 - val_loss: 0.8667 - val_accuracy: 0.6866
Epoch 5/10
437/437 [==============================] - 10s 23ms/step - loss: 0.4035 - accuracy: 0.8521 - val_loss: 0.7340 - val_accuracy: 0.7428
Epoch 6/10
437/437 [==============================] - 10s 23ms/step - loss: 0.3387 - accuracy: 0.8758 - val_loss: 0.8310 - val_accuracy: 0.7122
Epoch 7/10
437/437 [==============================] - 10s 23ms/step - loss: 0.3081 - accuracy: 0.8885 - val_loss: 0.9420 - val_accuracy:

Epoch 1/10
437/437 [==============================] - 10s 23ms/step - loss: 1.7872 - accuracy: 0.5050 - val_loss: 0.8566 - val_accuracy: 0.6509
Epoch 2/10
437/437 [==============================] - 10s 23ms/step - loss: 0.8309 - accuracy: 0.6869 - val_loss: 0.7196 - val_accuracy: 0.7147
Epoch 3/10
437/437 [==============================] - 10s 23ms/step - loss: 0.6543 - accuracy: 0.7555 - val_loss: 0.7025 - val_accuracy: 0.7263
Epoch 4/10
437/437 [==============================] - 10s 23ms/step - loss: 0.5517 - accuracy: 0.7958 - val_loss: 0.8308 - val_accuracy: 0.6720
Epoch 5/10
437/437 [==============================] - 10s 23ms/step - loss: 0.4422 - accuracy: 0.8379 - val_loss: 0.7009 - val_accuracy: 0.7413
Epoch 6/10
437/437 [==============================] - 10s 22ms/step - loss: 0.3612 - accuracy: 0.8684 - val_loss: 0.7800 - val_accuracy: 0.7157
Epoch 7/10
437/437 [==============================] - 10s 23ms/step - loss: 0.3223 - accuracy: 0.8821 - val_loss: 0.7803 - val_accuracy:

Epoch 1/10
437/437 [==============================] - 11s 24ms/step - loss: 1.6780 - accuracy: 0.5270 - val_loss: 0.8407 - val_accuracy: 0.6680
Epoch 2/10
437/437 [==============================] - 10s 23ms/step - loss: 0.7568 - accuracy: 0.7149 - val_loss: 0.7447 - val_accuracy: 0.7132
Epoch 3/10
437/437 [==============================] - 10s 24ms/step - loss: 0.5948 - accuracy: 0.7770 - val_loss: 0.6563 - val_accuracy: 0.7549
Epoch 4/10
437/437 [==============================] - 10s 24ms/step - loss: 0.4802 - accuracy: 0.8217 - val_loss: 0.8880 - val_accuracy: 0.6620
Epoch 5/10
437/437 [==============================] - 10s 24ms/step - loss: 0.4092 - accuracy: 0.8485 - val_loss: 0.6865 - val_accuracy: 0.7584
Epoch 6/10
437/437 [==============================] - 10s 24ms/step - loss: 0.3666 - accuracy: 0.8660 - val_loss: 0.8051 - val_accuracy: 0.7042
Epoch 7/10
437/437 [==============================] - 10s 24ms/step - loss: 0.3004 - accuracy: 0.8905 - val_loss: 0.7818 - val_accuracy:

Epoch 1/10
437/437 [==============================] - 10s 23ms/step - loss: 1.5942 - accuracy: 0.5334 - val_loss: 1.0358 - val_accuracy: 0.5681
Epoch 2/10
437/437 [==============================] - 10s 23ms/step - loss: 0.8239 - accuracy: 0.6871 - val_loss: 0.7836 - val_accuracy: 0.6786
Epoch 3/10
437/437 [==============================] - 10s 23ms/step - loss: 0.6376 - accuracy: 0.7602 - val_loss: 0.6743 - val_accuracy: 0.7559
Epoch 4/10
437/437 [==============================] - 10s 23ms/step - loss: 0.5097 - accuracy: 0.8106 - val_loss: 0.8846 - val_accuracy: 0.6801
Epoch 5/10
437/437 [==============================] - 10s 23ms/step - loss: 0.4169 - accuracy: 0.8471 - val_loss: 0.7451 - val_accuracy: 0.7423
Epoch 6/10
437/437 [==============================] - 10s 23ms/step - loss: 0.3472 - accuracy: 0.8736 - val_loss: 0.9127 - val_accuracy: 0.7107
Epoch 7/10
437/437 [==============================] - 10s 23ms/step - loss: 0.2999 - accuracy: 0.8908 - val_loss: 0.8757 - val_accuracy:

...

Epoch 1/10
437/437 [==============================] - 12s 27ms/step - loss: 1.4126 - accuracy: 0.4555 - val_loss: 0.8573 - val_accuracy: 0.6670
Epoch 2/10
437/437 [==============================] - 12s 27ms/step - loss: 0.8204 - accuracy: 0.6855 - val_loss: 0.7026 - val_accuracy: 0.7268
Epoch 3/10
437/437 [==============================] - 12s 27ms/step - loss: 0.6494 - accuracy: 0.7557 - val_loss: 0.7261 - val_accuracy: 0.7177
Epoch 4/10
437/437 [==============================] - 12s 27ms/step - loss: 0.4740 - accuracy: 0.8234 - val_loss: 0.9205 - val_accuracy: 0.6786
Epoch 5/10
437/437 [==============================] - 12s 27ms/step - loss: 0.3864 - accuracy: 0.8578 - val_loss: 0.9325 - val_accuracy: 0.6806
Epoch 6/10
437/437 [==============================] - 12s 27ms/step - loss: 0.3141 - accuracy: 0.8835 - val_loss: 1.0260 - val_accuracy: 0.6605
Epoch 7/10
437/437 [==============================] - 12s 26ms/step - loss: 0.2798 - accuracy: 0.8989 - val_loss: 0.9288 - val_accuracy:

Epoch 1/10
437/437 [==============================] - 12s 26ms/step - loss: 1.4600 - accuracy: 0.4491 - val_loss: 0.9669 - val_accuracy: 0.6620
Epoch 2/10
437/437 [==============================] - 11s 26ms/step - loss: 0.8955 - accuracy: 0.6537 - val_loss: 0.8025 - val_accuracy: 0.6921
Epoch 3/10
437/437 [==============================] - 12s 26ms/step - loss: 0.7297 - accuracy: 0.7211 - val_loss: 0.8312 - val_accuracy: 0.6570
Epoch 4/10
437/437 [==============================] - 12s 26ms/step - loss: 0.6087 - accuracy: 0.7731 - val_loss: 0.8217 - val_accuracy: 0.6966
Epoch 5/10
437/437 [==============================] - 12s 26ms/step - loss: 0.4886 - accuracy: 0.8179 - val_loss: 0.8637 - val_accuracy: 0.6620
Epoch 6/10
437/437 [==============================] - 12s 27ms/step - loss: 0.4154 - accuracy: 0.8470 - val_loss: 0.9863 - val_accuracy: 0.6344
Epoch 7/10
437/437 [==============================] - 12s 27ms/step - loss: 0.3693 - accuracy: 0.8648 - val_loss: 0.8741 - val_accuracy:

Epoch 1/10
437/437 [==============================] - 12s 26ms/step - loss: 1.3357 - accuracy: 0.4833 - val_loss: 0.8950 - val_accuracy: 0.6565
Epoch 2/10
437/437 [==============================] - 11s 25ms/step - loss: 0.8054 - accuracy: 0.6906 - val_loss: 0.7524 - val_accuracy: 0.7117
Epoch 3/10
437/437 [==============================] - 11s 26ms/step - loss: 0.6342 - accuracy: 0.7589 - val_loss: 0.7297 - val_accuracy: 0.7243
Epoch 4/10
437/437 [==============================] - 11s 26ms/step - loss: 0.5067 - accuracy: 0.8118 - val_loss: 0.8225 - val_accuracy: 0.6921
Epoch 5/10
437/437 [==============================] - 11s 25ms/step - loss: 0.4212 - accuracy: 0.8443 - val_loss: 0.8494 - val_accuracy: 0.6936
Epoch 6/10
437/437 [==============================] - 11s 26ms/step - loss: 0.3587 - accuracy: 0.8688 - val_loss: 1.1098 - val_accuracy: 0.6735
Epoch 7/10
437/437 [==============================] - 11s 26ms/step - loss: 0.3147 - accuracy: 0.8861 - val_loss: 0.8241 - val_accuracy:

Epoch 1/10
437/437 [==============================] - 11s 25ms/step - loss: 1.9889 - accuracy: 0.1416 - val_loss: 1.9458 - val_accuracy: 0.1115
Epoch 2/10
437/437 [==============================] - 11s 25ms/step - loss: 1.9460 - accuracy: 0.1430 - val_loss: 1.9412 - val_accuracy: 0.6735
Epoch 3/10
437/437 [==============================] - 11s 25ms/step - loss: 1.9460 - accuracy: 0.1406 - val_loss: 1.9451 - val_accuracy: 0.0090
Epoch 4/10
437/437 [==============================] - 11s 25ms/step - loss: 1.9460 - accuracy: 0.1420 - val_loss: 1.9398 - val_accuracy: 0.6735
Epoch 5/10
437/437 [==============================] - 11s 25ms/step - loss: 1.9460 - accuracy: 0.1422 - val_loss: 1.9467 - val_accuracy: 0.0477
Epoch 6/10
437/437 [==============================] - 11s 25ms/step - loss: 1.9460 - accuracy: 0.1402 - val_loss: 1.9417 - val_accuracy: 0.1115
Epoch 7/10
437/437 [==============================] - 11s 25ms/step - loss: 1.9460 - accuracy: 0.1390 - val_loss: 1.9402 - val_accuracy:

Epoch 1/10
437/437 [==============================] - 11s 25ms/step - loss: 1.4096 - accuracy: 0.4808 - val_loss: 0.9278 - val_accuracy: 0.6168
Epoch 2/10
437/437 [==============================] - 11s 25ms/step - loss: 0.8047 - accuracy: 0.6928 - val_loss: 0.7395 - val_accuracy: 0.7177
Epoch 3/10
437/437 [==============================] - 11s 25ms/step - loss: 0.6419 - accuracy: 0.7574 - val_loss: 0.7694 - val_accuracy: 0.7097
Epoch 4/10
437/437 [==============================] - 11s 25ms/step - loss: 0.5030 - accuracy: 0.8100 - val_loss: 0.9100 - val_accuracy: 0.6479
Epoch 5/10
437/437 [==============================] - 11s 25ms/step - loss: 0.4130 - accuracy: 0.8452 - val_loss: 0.8828 - val_accuracy: 0.7042
Epoch 6/10
437/437 [==============================] - 11s 25ms/step - loss: 0.3468 - accuracy: 0.8712 - val_loss: 0.8193 - val_accuracy: 0.7187
Epoch 7/10
437/437 [==============================] - 11s 25ms/step - loss: 0.3141 - accuracy: 0.8855 - val_loss: 1.0531 - val_accuracy:

# Step 7: Output Results

In [25]:
import csv

clean_table = []

for key in clean_dict:
    clean_table.append([key[0], key[1], key[2], clean_dict[key][0], clean_dict[key][1]])

with open('clean_mixed.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(clean_table)
    

In [26]:

oversample_table = []

for key in oversample_dict:
    oversample_table.append([key[0], key[1], key[2], oversample_dict[key][0], oversample_dict[key][1]])

with open('oversample_mixed.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(oversample_table)
    

In [27]:

image_augm_table = []

for key in image_augm_dict:
    image_augm_table.append([key[0], key[1], key[2], image_augm_dict[key][0], image_augm_dict[key][1]])

with open('image_augm_mixed.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(image_augm_table)
    

In [28]:

image_augm_over_table = []

for key in image_augm_over_dict:
    image_augm_over_table.append([key[0], key[1], key[2], image_augm_over_dict[key][0], image_augm_over_dict[key][1]])

with open('image_augm_over_mixed.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(image_augm_over_table)
    